# 1. Загрузка данных и первоначальный анализ распределения классов

Загружаем набор данных и визуализируем распределение классов (FAKE и REAL) для оценки баланса.

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline


import plotly.graph_objects as go
from plotly.subplots import make_subplots

import re
import joblib

COLORS = {
    'FAKE': '#FF6B6B',
    'REAL': '#4ECDC4',
    'primary': '#667eea',
    'secondary': '#764ba2',
}

FILE_PATH = 'https://storage.yandexcloud.net/academy.ai/practica/fake_news.csv'

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', '', text) # HTML
    text = re.sub(r'&[a-z]+;', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # URL
    text = re.sub(r'\[.*?\]', '', text) # текст в квадратных скобках
    text = re.sub(r'\(.*?\)', '', text) # текст в круглых скобках
    # эмодзи
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    punctuations_and_specials = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”‘’«»„”…–—""" # знаки препинания и специальные символы
    text = re.sub(r'[%s]' % re.escape(punctuations_and_specials), '', text)
    text = re.sub(r'\w*\d\w*', '', text)  # слова, содержащие цифры
    text = re.sub(r'\d+', '', text)  # цифры
    text = re.sub(r'\s+', ' ', text).strip()  # лишние пробелы
    return text

data = pd.read_csv(FILE_PATH)

In [14]:
class_counts = data['label'].value_counts()
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'pie'}, {'type': 'xy'}]])
fig.add_trace(go.Pie(values=class_counts.values, labels=class_counts.index,
                     marker_colors=[COLORS[l] for l in class_counts.index],
                     hole=0.3, textinfo='label+percent'), row=1, col=1)
fig.add_trace(go.Bar(x=class_counts.index, y=class_counts.values, text=class_counts.values,
                     marker_color=[COLORS[l] for l in class_counts.index]), row=1, col=2)
fig.update_layout(width=800, height=400, title_text="<b>Распределение 'label'</b>",
                  title_x=0.5, showlegend=False)
fig.show()

 Анализ показывает, что классы FAKE (49.9%) и REAL (50.1%) хорошо сбалансированы

# 2. Предобработка данных

- Выполняется очистка текстовых полей (title и text) с использованием регулярных выражений для удаления HTML-тегов, URL, эмодзи, пунктуации, цифр и лишних пробелов. Затем очищенные заголовок и текст объединяются в новый столбец content, который будет использоваться для извлечения признаков. Дубликаты и пропущенные значения удаляются для обеспечения чистоты данных.

In [15]:
data = data.drop(columns=['Unnamed: 0'], errors='ignore')
data = data.drop_duplicates()
data = data.dropna()
data['title'] = data['title'].fillna('').astype(str)
data['text'] = data['text'].fillna('').astype(str)

data['title'] = data['title'].apply(clean_text)
data['text'] = data['text'].apply(clean_text)

data['content'] = data['title'] + ' ' + data['text']

X = data['content']
y = data['label']

print(f"Размер датасета после предобработки: {data.shape}")

Размер датасета после предобработки: (6306, 4)


In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=24
)

# 3. Создание и оценка модели

Инициализируется пайплайн, состоящий из двух ключевых компонентов:
- __TfidfVectorizer__: Используется для преобразования текстовых данных в TF-IDF векторы.
- __PassiveAggressiveClassifier__: Выступает в качестве классификатора.

Модель оценивается с использованием 10-кратной кросс-валидации.

In [ ]:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(
      ngram_range=(1,2),
      sublinear_tf=True,
      stop_words='english',
      min_df=3,
    )),
    ('classifier', PassiveAggressiveClassifier(
        C=2,
        loss='squared_hinge',
        random_state=42,
    ))
])

n_splits=10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cv_accuracy_scores = cross_val_score(
    pipeline,
    X_train, y_train,
    cv=kf,
    scoring='accuracy',
    n_jobs=-1
)

y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=kf, n_jobs=-1)

In [89]:
import numpy as np
acc = accuracy_score(y_train, y_pred_cv)
report = classification_report(y_train, y_pred_cv, output_dict=True)
cm = confusion_matrix(y_train, y_pred_cv, labels=['FAKE', 'REAL'])

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "xy"}],
           [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=(
        f"Средняя точность CV ({acc:.1%})",
        f"Точность по {n_splits} фолдам",
        "Матрица ошибок (CV)",
        "Метрики по классам (CV)"
    ),
    vertical_spacing=0.12, horizontal_spacing=0.15
)

fig.add_trace(go.Indicator(
    mode="gauge+number", value=acc * 100,
    gauge={'axis': {'range': [80, 100], 'tickwidth': 1, 'dtick': 5},
           'bar': {'color': COLORS['primary'], 'thickness': 0.7},
           'steps': [{'range': [80, 90], 'color': COLORS['FAKE']},
                    {'range': [90, 100], 'color': COLORS['REAL']}],
           'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 90}}
), row=1, col=1)

mean_acc = np.mean(cv_accuracy_scores)
std_acc = np.std(cv_accuracy_scores)
fig.add_trace(go.Scatter(
    x=list(range(1, len(cv_accuracy_scores) + 1)), y=cv_accuracy_scores,
    mode='markers+lines', name="Точность фолда",
    marker=dict(color=[COLORS['REAL'] if s >= mean_acc else COLORS['FAKE']
                      for s in cv_accuracy_scores], size=8),
    line=dict(color='gray', width=2),
    text=[f'{s:.1%}' for s in cv_accuracy_scores],
    textposition='top center'
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=list(range(1, len(cv_accuracy_scores) + 1)),
    y=[mean_acc] * len(cv_accuracy_scores),
    mode='lines', name=f"Среднее: {mean_acc:.1%}",
    line=dict(dash='dash', color='blue', width=2),
    showlegend=False
), row=1, col=2)

fig.update_yaxes(range=[min(cv_accuracy_scores)*0.95, max(cv_accuracy_scores)*1.02],
                title_text="Точность", row=1, col=2)

cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig.add_trace(go.Heatmap(
    z=cm_norm, x=['FAKE', 'REAL'], y=['FAKE', 'REAL'],
    text=[[f'{cm[i,j]}<br>({cm_norm[i,j]:.1%})' for j in range(2)] for i in range(2)],
    texttemplate='%{text}', colorscale='Blues',
    hovertemplate="Истина: %{y}<br>Предсказание: %{x}<br>%{text}<extra></extra>"
), row=2, col=1)

metrics = ['precision', 'recall', 'f1-score']
classes = ['FAKE', 'REAL']
colors = [COLORS['FAKE'], COLORS['REAL']]

for i, cls in enumerate(classes):
    vals = [report[cls][m] for m in metrics]
    fig.add_trace(go.Bar(
        x=metrics, y=vals, name=cls, marker_color=colors[i],
        text=[f'{v:.3f}' for v in vals], textposition='outside',
        error_y=dict(type='constant', value=0.01, visible=True)
    ), row=2, col=2)

fig.update_yaxes(range=[0, 1.05], title_text="Значение", row=2, col=2)

fig.update_layout(
    height=800, width=1200,
    title={'text': f'<b>Кросс-валидация</b>',
           'x': 0.5, 'font': {'size': 18}},
    showlegend=True,
    legend=dict(x=0.99, y=0.01, xanchor='right', yanchor='bottom'),
    barmode='group'
)

fig.show()

# 4.Обучение и сохранение финальной модели

In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [92]:
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
cm = confusion_matrix(y_test, y_pred, labels=['FAKE', 'REAL'])

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "xy"}], [{"type": "xy"}, {"type": "table"}]],
    subplot_titles=("Общая точность", "Метрики по классам", "Матрица ошибок", "Сводная таблица"),
    vertical_spacing=0.12, horizontal_spacing=0.1
)

fig.add_trace(go.Indicator(
    mode="gauge+number", value=acc * 100,
    gauge={'axis': {'range': [80, 100], 'tickwidth': 1, 'dtick': 5},
           'bar': {'color': COLORS['primary'], 'thickness': 0.7},
           'steps': [{'range': [80, 90], 'color': COLORS['FAKE']},
                    {'range': [90, 100], 'color': COLORS['REAL']}],
           'threshold': {'line': {'color': "red", 'width': 3},
                        'thickness': 0.75, 'value': 90}}
), row=1, col=1)


cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig.add_trace(go.Heatmap(
    z=cm_norm, x=['Предсказано FAKE', 'Предсказано REAL'],
    y=['Истина FAKE', 'Истина REAL'],
    text=[[f'{cm[i,j]}<br>({cm_norm[i,j]:.1%})' for j in range(2)] for i in range(2)],
    texttemplate='%{text}', colorscale='Blues', zmin=0, zmax=1,
    hovertemplate="Истина: %{y}<br>Предсказано: %{x}<br>Доля: %{z:.1%}<extra></extra>"
), row=2, col=1)

for i, cls in enumerate(['FAKE', 'REAL']):
    vals = [report[cls][m] for m in ['precision', 'recall', 'f1-score']]
    fig.add_trace(go.Bar(
        x=['Precision', 'Recall', 'F1-Score'], y=vals, name=cls,
        marker_color=COLORS[cls], text=[f'{v:.2f}' for v in vals],
        textposition='outside', yaxis='y2'
    ), row=1, col=2)

table_data = [
    ['Метрика', 'FAKE', 'REAL', 'Macro Avg', 'Weighted Avg'],
    ['Precision', f"{report['FAKE']['precision']:.2f}", f"{report['REAL']['precision']:.2f}",
     f"{report['macro avg']['precision']:.2f}", f"{report['weighted avg']['precision']:.2f}"],
    ['Recall', f"{report['FAKE']['recall']:.2f}", f"{report['REAL']['recall']:.2f}",
     f"{report['macro avg']['recall']:.2f}", f"{report['weighted avg']['recall']:.2f}"],
    ['F1-Score', f"{report['FAKE']['f1-score']:.2f}", f"{report['REAL']['f1-score']:.2f}",
     f"{report['macro avg']['f1-score']:.2f}", f"{report['weighted avg']['f1-score']:.2f}"]
]

fig.add_trace(go.Table(
    header=dict(values=table_data[0], fill_color='lightblue', font=dict(size=12)),
    cells=dict(values=list(zip(*table_data[1:])), fill_color='white', font=dict(size=11))
), row=2, col=2)

fig.update_yaxes(range=[0, 1.05], tickformat='.1%', title="Значение", row=1, col=2)
fig.update_xaxes(title="Метрики", row=1, col=2)
fig.update_layout(
    height=800, width=1200, title="Анализ производительности модели",
    showlegend=False, barmode='group', font=dict(size=11)
)


In [ ]:
joblib.dump(pipeline, 'fake_news_detect.joblib')

['fake_news_detect_model.joblib']